In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import json

# --- 1. 환경 설정 및 경로 지정 ---
# 🚨 사용자 설정: 데이터셋 폴더 경로를 수정하세요.
DATA_DIR = './data/image/'
IMAGE_SIZE = 224
BATCH_SIZE = 32
RANDOM_SEED = 42
EPOCHS = 40  # 총 에포크 (Phase 1 + Phase 2)
MODEL_NAME = 'acne_classifier_efficientnetb0'
model_path = f'{MODEL_NAME}_best.keras'

# 4개 클래스 정의 (Normal 제외)
CLASSES = ['comedones', 'pustules', 'papules', 'folliculitis']
num_classes = len(CLASSES)

print(f"모델 학습을 위한 {num_classes}개 클래스: {CLASSES}")
print("---" * 20)

# --- 2. 데이터 로드 및 분할 (불균형 해소 포함) ---

X_paths = []  # 이미지 경로 저장 리스트
Y_labels = [] # 라벨 인덱스 저장 리스트

print("데이터셋 파일 경로 로드 중...")
for class_index, class_name in enumerate(CLASSES):
    class_path = os.path.join(DATA_DIR, class_name)
    if not os.path.isdir(class_path):
        print(f"경고: 경로를 찾을 수 없습니다: {class_path}")
        continue
    for img_name in os.listdir(class_path):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            X_paths.append(os.path.join(class_path, img_name))
            Y_labels.append(class_index)

X_paths = np.array(X_paths)
Y_labels = np.array(Y_labels)

if len(X_paths) == 0:
    print("🚨 오류: 데이터셋 폴더에서 이미지를 찾을 수 없습니다. 경로를 확인하세요.")
    exit()

# 1. Stratified Split: 클래스 비율을 유지하며 학습/검증/테스트 데이터 분할
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X_paths, Y_labels, test_size=0.3, random_state=RANDOM_SEED, stratify=Y_labels
)
X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED, stratify=Y_temp
)

# 2. 클래스 가중치 계산 (학습 불균형 보정)
class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(Y_train),
    y=Y_train
)
class_weights = dict(enumerate(class_weights_array))

print(f"\n📊 데이터셋 요약: 전체 {len(X_paths)}장")
print(f"  - 학습 데이터: {len(X_train)}장")
print(f"  - 검증 데이터: {len(X_val)}장")
print(f"  - 테스트 데이터: {len(X_test)}장")
print("\n⚖️ 계산된 클래스 가중치:")
for i, weight in class_weights.items():
    print(f"  - {CLASSES[i]}: {weight:.3f}")
print("---" * 20)


# 3. Keras Generator용 DataFrame 생성
def create_dataframe(X_paths, Y_indices, classes):
    df = pd.DataFrame({'filename': X_paths, 'class_index': Y_indices})
    df['class'] = df['class_index'].apply(lambda x: classes[x])
    return df

train_df = create_dataframe(X_train, Y_train, CLASSES)
val_df = create_dataframe(X_val, Y_val, CLASSES)

# 4. 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df, x_col='filename', y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, class_mode='categorical', seed=RANDOM_SEED
)
val_generator = val_datagen.flow_from_dataframe(
    val_df, x_col='filename', y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, class_mode='categorical', seed=RANDOM_SEED
)


# --- 3. 모델 정의 및 콜백 설정 ---

def build_model(num_classes):
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
    )

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = build_model(num_classes)

# 콜백 설정 (ModelCheckpoint를 통한 최적 모델 저장 필수)
callbacks = [
    # 1. 가장 높은 val_accuracy를 기록한 시점의 모델 가중치 저장
    ModelCheckpoint(
        filepath=model_path,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    # 2. val_loss가 8번의 에포크 동안 개선되지 않으면 학습 종료 (patience 완화)
    EarlyStopping(
        monitor='val_loss',
        patience=8,
        mode='min',
        verbose=1,
        restore_best_weights=True # 최적 가중치 복구
    ),
    # 3. val_loss가 5번의 에포크 동안 개선되지 않으면 학습률 감소
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=0.00001,
        verbose=1
    )
]

# --- 4. 학습 시작: Phase 1 (Feature Extraction) ---

print("\n--- Phase 1: Feature Extraction (EfficientNet 동결) 시작 ---")

# EfficientNet 레이어 동결
for layer in model.layers:
    layer.trainable = False
# 최상단 커스텀 레이어만 학습 (마지막 Dense 레이어)
for layer in model.layers[-4:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), # 비교적 높은 학습률
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_phase1 = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10, # 초기 10 에포크
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    callbacks=callbacks,
    class_weight=class_weights, # 클래스 가중치 적용
    verbose=1
)

# --- 5. 학습 시작: Phase 2 (Fine-Tuning) ---

print("\n--- Phase 2: Fine-Tuning (전체 EfficientNet 레이어 해동) 시작 ---")

# 전체 레이어 동결 해제
for layer in model.layers:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # 매우 낮은 학습률
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_phase2 = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS, # 총 에포크까지
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    callbacks=callbacks,
    class_weight=class_weights, # 클래스 가중치 적용
    initial_epoch=history_phase1.epoch[-1] + 1, # Phase 1 끝난 다음 에포크부터 시작
    verbose=1
)

# --- 6. 최종 평가 및 리포트 ---

print("\n--- 최종 모델 로드 및 테스트 데이터 평가 ---")

# 가장 성능이 좋았던 모델 로드
model.load_weights(model_path)

test_df = create_dataframe(X_test, Y_test, CLASSES)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, x_col='filename', y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

val_loss, val_acc = model.evaluate(test_generator, verbose=1)

# 예측 및 리포트
Y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(Y_pred, axis=1)

print("\n" + "=" * 70)
print(" " * 20 + "최종 테스트 결과 요약")
print("=" * 70)
print(f"\n🎯 모델 성능:")
print(f"  - 최종 Test Accuracy: {val_acc:.4f}")
print(f"  - 최종 Test Loss: {val_loss:.4f}")

# 분류 리포트 출력 (Precision, Recall, F1-Score 확인)
print("\nClassification Report:\n")
print(classification_report(test_generator.classes, y_pred_classes, target_names=CLASSES))

# 혼동 행렬 출력 (모델의 오분류 패턴 확인)
conf_matrix = confusion_matrix(test_generator.classes, y_pred_classes)
print("\nConfusion Matrix:\n")
print(conf_matrix)

# 최종 학습 정보 저장 (선택 사항)
model_info = {
    'classes': CLASSES,
    'test_accuracy': float(val_acc),
    'test_loss': float(val_loss),
    'class_weights': class_weights,
    'model_path': model_path
}
with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=4)
print(f"\n📁 최종 모델이 {model_path}에 저장되었습니다.")

모델 학습을 위한 4개 클래스: ['comedones', 'pustules', 'papules', 'folliculitis']
------------------------------------------------------------
데이터셋 파일 경로 로드 중...

📊 데이터셋 요약: 전체 2228장
  - 학습 데이터: 1559장
  - 검증 데이터: 334장
  - 테스트 데이터: 335장

⚖️ 계산된 클래스 가중치:
  - comedones: 2.165
  - pustules: 1.107
  - papules: 0.430
  - folliculitis: 3.248
------------------------------------------------------------
Found 1559 validated image filenames belonging to 4 classes.
Found 334 validated image filenames belonging to 4 classes.

--- Phase 1: Feature Extraction (EfficientNet 동결) 시작 ---


/Users/jeongjin/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.2287 - loss: 1.7015
Epoch 1: val_accuracy improved from -inf to 0.22187, saving model to acne_classifier_efficientnetb0_best.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 22s 382ms/step - accuracy: 0.2288 - loss: 1.7001 - val_accuracy: 0.2219 - val_loss: 1.7427 - learning_rate: 0.0010
Epoch 2/10
 1/48 ━━━━━━━━━━━━━━━━━━━━ 12s 259ms/step - accuracy: 0.1875 - loss: 1.5932

/Users/jeongjin/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.22187 to 0.23125, saving model to acne_classifier_efficientnetb0_best.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.1875 - loss: 1.5932 - val_accuracy: 0.2313 - val_loss: 1.6842 - learning_rate: 0.0010
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step - accuracy: 0.2225 - loss: 1.5262
Epoch 3: val_accuracy did not improve from 0.23125
48/48 ━━━━━━━━━━━━━━━━━━━━ 20s 404ms/step - accuracy: 0.2226 - loss: 1.5266 - val_accuracy: 0.2188 - val_loss: 1.6306 - learning_rate: 0.0010
Epoch 4/10
 1/48 ━━━━━━━━━━━━━━━━━━━━ 15s 330ms/step - accuracy: 0.1562 - loss: 1.4825
Epoch 4: val_accuracy did not improve from 0.23125
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.1562 - loss: 1.4825 - val_accuracy: 0.2281 - val_loss: 1.5963 - learning_rate: 0.0010
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.2209 - loss: 1.5635
Epoch 5: val_accuracy did not improve from 0.23125
48/48 ━━━━━━━━━━━━━━━━━━━━ 18s 369ms/step - accurac

/Users/jeongjin/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 249ms/step - accuracy: 0.6058 - loss: 0.9885
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 323ms/step

                    최종 테스트 결과 요약

🎯 모델 성능:
  - 최종 Test Accuracy: 0.5731
  - 최종 Test Loss: 1.0007

Classification Report:

              precision    recall  f1-score   support

   comedones       0.39      0.51      0.44        39
    pustules       0.00      0.00      0.00        25
     papules       0.88      0.58      0.70       195
folliculitis       0.37      0.76      0.50        76

    accuracy                           0.57       335
   macro avg       0.41      0.47      0.41       335
weighted avg       0.64      0.57      0.58       335


Confusion Matrix:

[[ 20   0   3  16]
 [  9   0   3  13]
 [ 13   0 114  68]
 [  9   0   9  58]]

📁 최종 모델이 acne_classifier_efficientnetb0_best.keras에 저장되었습니다.


/Users/jeongjin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeongjin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeongjin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le